In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from train_utils import *

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=8264, hidden_size=4096):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, 1024, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(3, 1)
        self.fc2 = nn.Linear(1, input_size)

    def forward(self, x):
        batch_size = x.size(0)
        out, _ = self.lstm1(x)
        out = self.fc1(out[:, -1, :])
        out = nn.functional.relu(out)
        out = torch.tanh(self.fc2(out))

        return out

In [ ]:
data_dir = "/sdf/group/lcls/ds/scratch/s2e_scratch/Data/SFG_reIm/version1"
train_dataset = CustomSequence(data_dir, range(80), file_batch_size=1, model_batch_size=512)
val_dataset = CustomSequence(data_dir, range(80, 81), file_batch_size=1, model_batch_size=512)


In [ ]:
model = LSTMModel()

In [ ]:
train(model, train_dataset, num_epochs=10, val_dataset=val_dataset, use_gpu=True, data_parallel=True, out_dir=".", model_name="model", verbose=1, save_checkpoints=True, custom_loss=None)

In [ ]:
test_dataset = CustomSequence(data_dir, range(90, 92), file_batch_size=1, model_batch_size=512, test_mode=True)

In [ ]:
predict(model, model_param_path="model_epoch_2.pth", test_dataset=test_dataset, use_gpu=True, data_parallel=False, output_dir=".", output_name="all_preds.npy", verbose=1)